# Deciphering Code with Character-Level RNN





## Dataset

In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv('./data/CaesarCipher.csv')
df

,Passwords,ciphertext,key
0,123456,567890,6
1,12345,56789,6
2,123456789,789012345,14
3,password,bmeeiadp,14
4,iloveyou,nqtajdtz,21
...,...,...,...
14343471,"xCvBnM,",tYrXjI,4
14343472,ie168,bx491,7
14343473,abygurl69,cdaiwtn69,24
14343474,a6_123,i8345,18


In [3]:
df = df.dropna()[:120000]

In [4]:
df['Passwords'].apply(str)
df['ciphertext'].apply(str)
df

,Passwords,ciphertext,key
0,123456,567890,6
1,12345,56789,6
2,123456789,789012345,14
3,password,bmeeiadp,14
4,iloveyou,nqtajdtz,21
...,...,...,...
120022,270796,947463,3
120023,270783,947450,13
120024,27061992,16950881,1
120025,27061985,83627541,14


## Preprocessing Data

In [5]:
def tokenize(x):
    x_tk = Tokenizer(char_level=True)
    x_tk.fit_on_texts(x)                 

    return x_tk.texts_to_sequences(x), x_tk

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    
    return pad_sequences(x, maxlen=length, padding="post", truncating="post",)

### Preprocess Pipeline

In [6]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [7]:
preproc_code_sentences, preproc_plaintext_sentences, code_tokenizer, plaintext_tokenizer = preprocess(df['Passwords'], df['ciphertext'])

print('Data Preprocessed')

Data Preprocessed


In [8]:
preproc_code_sentences[0]

array([ 3, 12, 22, 26, 27, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0])

In [9]:
len(code_tokenizer.word_index)+1

78

In [10]:
len(plaintext_tokenizer.word_index)+1

41

In [11]:
plaintext_tokenizer.word_index

{'1': 1,
 'x': 2,
 'w': 3,
 'j': 4,
 'f': 5,
 'z': 6,
 'q': 7,
 'k': 8,
 'v': 9,
 'h': 10,
 'y': 11,
 'b': 12,
 'g': 13,
 'c': 14,
 'd': 15,
 'u': 16,
 'p': 17,
 't': 18,
 'm': 19,
 'l': 20,
 'n': 21,
 's': 22,
 'i': 23,
 'r': 24,
 '0': 25,
 'o': 26,
 'e': 27,
 '2': 28,
 'a': 29,
 '9': 30,
 '8': 31,
 '3': 32,
 '4': 33,
 '6': 34,
 '5': 35,
 '7': 36,
 'ã': 37,
 'à': 38,
 'â': 39,
 '¹': 40}

In [12]:
preproc_code_sentences.shape

(120000, 39)

In [13]:
preproc_plaintext_sentences.shape

(120000, 32, 1)

# LSTM

In [14]:
from keras.layers import GRU, Input, Dense, TimeDistributed, RNN, LSTM
from keras.models import Model, Sequential
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow


def simple_model(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = LSTM(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

tmp_x = pad(preproc_code_sentences, preproc_plaintext_sentences.shape[1]) 
tmp_x = tmp_x.reshape((-1, preproc_plaintext_sentences.shape[-2], 1))     

In [15]:
tmp_x.shape

(120000, 32, 1)

In [16]:
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 1)]           0         
_________________________________________________________________
Layer1 (LSTM)                (None, 32, 64)            16896     
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 41)            2665      
Total params: 19,561
Trainable params: 19,561
Non-trainable params: 0
_________________________________________________________________


In [17]:
simple_rnn_model.get_layer(name="Layer1").output.shape

TensorShape([None, 32, 64])

In [18]:
simple_rnn_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=32, epochs=15, validation_split=0.3, use_multiprocessing=True)

Epoch 1/15
2625/2625 [==============================] - 31s 6ms/step - loss: 0.9278 - accuracy: 0.7772 - val_loss: 0.7818 - val_accuracy: 0.7850
Epoch 2/15
2625/2625 [==============================] - 14s 5ms/step - loss: 0.7546 - accuracy: 0.7899 - val_loss: 0.7492 - val_accuracy: 0.7881
Epoch 3/15
2625/2625 [==============================] - 14s 5ms/step - loss: 0.7249 - accuracy: 0.7926 - val_loss: 0.7341 - val_accuracy: 0.7887
Epoch 4/15
2625/2625 [==============================] - 14s 5ms/step - loss: 0.7145 - accuracy: 0.7934 - val_loss: 0.7278 - val_accuracy: 0.7901
Epoch 5/15
2625/2625 [==============================] - 14s 5ms/step - loss: 0.7093 - accuracy: 0.7940 - val_loss: 0.7240 - val_accuracy: 0.7905
Epoch 6/15
2625/2625 [==============================] - 14s 5ms/step - loss: 0.7054 - accuracy: 0.7946 - val_loss: 0.7212 - val_accuracy: 0.7901
Epoch 7/15
2625/2625 [==============================] - 14s 5ms/step - loss: 0.7030 - accuracy: 0.7948 - val_loss: 0.7198 - val_ac

In [19]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], plaintext_tokenizer))

1 2 3 8 5 6 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [20]:
df['Passwords'][0]

'123456'

# GRU

In [21]:
def simple_model1(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = GRU(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)  # output must be batchsize x timesteps x units
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

In [22]:
simple_rnn_model1 = simple_model1(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 1)]           0         
_________________________________________________________________
Layer1 (GRU)                 (None, 32, 64)            12864     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 41)            2665      
Total params: 15,529
Trainable params: 15,529
Non-trainable params: 0
_________________________________________________________________


In [23]:
simple_rnn_model1.fit(tmp_x, preproc_plaintext_sentences, batch_size=32, epochs=15, validation_split=0.3, use_multiprocessing=True)

Epoch 1/15
2625/2625 [==============================] - 16s 5ms/step - loss: 0.9549 - accuracy: 0.7810 - val_loss: 0.7640 - val_accuracy: 0.7863
Epoch 2/15
2625/2625 [==============================] - 13s 5ms/step - loss: 0.7326 - accuracy: 0.7918 - val_loss: 0.7319 - val_accuracy: 0.7890
Epoch 3/15
2625/2625 [==============================] - 14s 5ms/step - loss: 0.7109 - accuracy: 0.7932 - val_loss: 0.7246 - val_accuracy: 0.7894
Epoch 4/15
2625/2625 [==============================] - 14s 5ms/step - loss: 0.7059 - accuracy: 0.7938 - val_loss: 0.7211 - val_accuracy: 0.7903
Epoch 5/15
2625/2625 [==============================] - 14s 5ms/step - loss: 0.7024 - accuracy: 0.7947 - val_loss: 0.7165 - val_accuracy: 0.7912
Epoch 6/15
2625/2625 [==============================] - 14s 5ms/step - loss: 0.6997 - accuracy: 0.7951 - val_loss: 0.7147 - val_accuracy: 0.7912
Epoch 7/15
2625/2625 [==============================] - 16s 6ms/step - loss: 0.6972 - accuracy: 0.7955 - val_loss: 0.7137 - val_ac

In [24]:
print(logits_to_text(simple_rnn_model1.predict(tmp_x[:1])[0], plaintext_tokenizer))

1 2 3 3 5 6 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [26]:
df['Passwords'][0]

'123456'

# SimpleRNN

In [27]:
from keras.layers import SimpleRNN 
def simple_model2(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])      
    seq = SimpleRNN(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)   
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)    
    model = Model(inputs = x, outputs = output)    
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])    
    model.summary()    
    return model

In [28]:
simple_rnn_model2 = simple_model2(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 1)]           0         
_________________________________________________________________
Layer1 (SimpleRNN)           (None, 32, 64)            4224      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 41)            2665      
Total params: 6,889
Trainable params: 6,889
Non-trainable params: 0
_________________________________________________________________


In [29]:
simple_rnn_model2.fit(tmp_x, preproc_plaintext_sentences, batch_size=32, epochs=15, validation_split=0.3, use_multiprocessing=True)

Epoch 1/15
2625/2625 [==============================] - 52s 18ms/step - loss: 0.8873 - accuracy: 0.7773 - val_loss: 0.7871 - val_accuracy: 0.7842
Epoch 2/15
2625/2625 [==============================] - 47s 18ms/step - loss: 0.7574 - accuracy: 0.7906 - val_loss: 0.7574 - val_accuracy: 0.7872
Epoch 3/15
2625/2625 [==============================] - 47s 18ms/step - loss: 0.7321 - accuracy: 0.7927 - val_loss: 0.7404 - val_accuracy: 0.7890
Epoch 4/15
2625/2625 [==============================] - 47s 18ms/step - loss: 0.7191 - accuracy: 0.7935 - val_loss: 0.7316 - val_accuracy: 0.7898
Epoch 5/15
2625/2625 [==============================] - 46s 17ms/step - loss: 0.7124 - accuracy: 0.7940 - val_loss: 0.7274 - val_accuracy: 0.7898
Epoch 6/15
2625/2625 [==============================] - 46s 18ms/step - loss: 0.7074 - accuracy: 0.7945 - val_loss: 0.7239 - val_accuracy: 0.7896
Epoch 7/15
2625/2625 [==============================] - 47s 18ms/step - loss: 0.7056 - accuracy: 0.7945 - val_loss: 0.7209 -

In [30]:
print(logits_to_text(simple_rnn_model2.predict(tmp_x[:1])[0], plaintext_tokenizer))

1 2 8 8 7 7 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [31]:
df['Passwords'][0]

'123456'